# Урок 2. Масштабирование признаков. L1 и L2 регуляризация. Стохастический градиентный спуск.

**1. Постройте график зависимости весов всех признаков от lambda в L2-регуляризации (на данных из урока). Сделайте вывод.**

**2. Можно ли к одному и тому же признаку применить сразу и нормализацию, и стандартизацию?**

**3. Напишите функцию наподобие eval_model_reg2, но для применения L1-регуляризации.**